# PIR - LSTM n°4 Hyperparameters Optimization

This notebook is used for hyperparameter optimization for the model implemented in LSTMn°3.ipynb

We aim to optimize the LSTM model's hyperparameters by using all the training data (2015 - 2022) for training and validation. Pour aller plus vite on ne prendra qu'un échantillon de stations étant donné que ça sélectionne les paramètres optiamuax en moyenne sur l'échantillon de stations.

## 1: Imports

In [1]:
# Custom utility functions used in the project
import utils

import os

# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error

from keras.models import Sequential
from keras.layers import Input, LSTM, Dense
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras import backend as K

2026-01-13 17:21:50.979203: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-13 17:21:51.028007: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-13 17:21:55.436641: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# imports
import optuna
import tqdm
from optuna.samplers import TPESampler

/home/pedoussa/.conda/envs/tf311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---

## 2: Hyperparameters Optimization

In [3]:
# Load the data
x = pd.read_csv('train_f_x.csv')
y = pd.read_csv('y_train_sncf.csv')
x_0 = pd.read_csv('x_test.csv')

In [4]:
# Data preparation
df_per_station_train, df_per_station_0 = utils.prepare_data(x, y, x_0, remove_covid=True)

Hyperparameter tuning is conducted on a subset of 50 stations (out of 439) to significantly reduce runtime, under the assumption that optimal hyperparameters generalize across stations.

In [11]:
# Sampling
sample_size = 30
seed = 30
stations_sample = utils.sample_stations(df_per_station_train, n=sample_size, seed=seed)

In [12]:
print(stations_sample.keys())

dict_keys(['CUF', 'KQC', 'J52', 'XTG', 'EOE', 'O2O', 'Y5A', 'NZV', '4RW', '488', 'WDP', 'ESA', 'GUQ', '0Y9', '0WC', 'KHT', 'FBH', 'I9U', '6WR', '94K', 'X3T', '9RF', '700', 'V2P', '1EJ', '8X9', 'VML', '4AH', 'VV4', 'KN2'])


In [ ]:
def objective(trial, sample_train, features):
    # HYPERPARAMETERS TO OPTIMIZE
    params = {
        "units": trial.suggest_int("units", 16, 128, step=16),
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True),
        "batch_size": trial.suggest_categorical("batch_size", [16, 32, 64]),
        "seq_len": trial.suggest_int("seq_len", 60, 120, step=10),
    }

    val_mse_scores = []
    idx = 1
    total_stations = len(sample_train)
    for name_station in sample_train:
        try:
            print(f'Station {name_station}, {idx}/{total_stations}')
            idx+=1
            # DATA EXTRACTION
            df_train = sample_train[name_station]
            X_raw = df_train[features]
            y_raw = df_train['y'].values.reshape(-1, 1)

            # SCALING
            scaler_X, scaler_y = MinMaxScaler(), MinMaxScaler()
            X_scaled = scaler_X.fit_transform(X_raw)
            y_scaled = scaler_y.fit_transform(y_raw)

            # CREATE UNIFORM SEQUENCES
            X_seq, y_seq = utils.create_sequences_uniform(
                pd.DataFrame(X_scaled), 
                pd.DataFrame(y_scaled), 
                params["seq_len"], 
                0.25
            )

            # RANDOM SPLIT BETWEEN LEARNING AND VALIDATION DATA
            # ========================================================
            X_train, X_val, y_train, y_val = train_test_split(
                X_seq, 
                y_seq, 
                test_size=0.2, # validation_size
                random_state=42,
                shuffle=True
            )

            # MODEL CONSTRUCTION
            # ========================================================
            model = Sequential([
                Input(shape=(params["seq_len"], X_raw.shape[1])),
                LSTM(units=params["units"], activation='tanh'),
                Dense(1)
            ])
            model.compile(optimizer=Adam(learning_rate=params["learning_rate"]), loss='mse')

            # MODEL TRAINING
            # ========================================================
            callback = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

            history = model.fit(X_train, y_train, validation_data=(X_val, y_val), 
                    epochs=30, batch_size=params["batch_size"], 
                    verbose=0, callbacks=[callback])

            # VALIDATION MAPE SCORE
            # ========================================================
            # Use the best validation loss from history (already normalized)
            best_val_mse = min(history.history['val_loss'])
            val_mse_scores.append(best_val_mse)
            
            K.clear_session()
            del model

        except Exception as e:
                    print(f"Station {name_station} skipped due to error {e}")

    return np.mean(val_mse_scores)

In [ ]:
# Execution
run = 0
if (run == 1):
    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler())
    study.optimize(lambda trial: objective(trial, stations_sample, ['job', 'ferie', 'vacances']), n_trials=20)
else:
    print("run == 0")

[I 2026-01-13 17:37:57,587] A new study created in memory with name: no-name-eb4d168e-56d1-446a-9c56-3888aa3d527b


Station CUF, 1/30
Station KQC, 2/30
Station J52, 3/30
Station XTG, 4/30
Station EOE, 5/30
Station O2O, 6/30
Station Y5A, 7/30
Station NZV, 8/30
Station 4RW, 9/30
Station 488, 10/30
Station WDP, 11/30
Station ESA, 12/30
Station GUQ, 13/30
Station 0Y9, 14/30
Station 0WC, 15/30
Station KHT, 16/30
Station FBH, 17/30
Station I9U, 18/30
Station 6WR, 19/30
Station 94K, 20/30
Station X3T, 21/30
Station 9RF, 22/30
Station 700, 23/30
Station V2P, 24/30
Station 1EJ, 25/30
Station 8X9, 26/30
Station VML, 27/30
Station 4AH, 28/30
Station VV4, 29/30
Station KN2, 30/30


[I 2026-01-13 17:40:02,055] Trial 0 finished with value: 0.03274989788575719 and parameters: {'units': 32, 'learning_rate': 0.0013066425346952345, 'batch_size': 32, 'seq_len': 60}. Best is trial 0 with value: 0.03274989788575719.


Station CUF, 1/30
Station KQC, 2/30
Station J52, 3/30
Station XTG, 4/30
Station EOE, 5/30
Station O2O, 6/30
Station Y5A, 7/30
Station NZV, 8/30
Station 4RW, 9/30
Station 488, 10/30
Station WDP, 11/30
Station ESA, 12/30
Station GUQ, 13/30
Station 0Y9, 14/30
Station 0WC, 15/30
Station KHT, 16/30
Station FBH, 17/30
Station I9U, 18/30
Station 6WR, 19/30
Station 94K, 20/30
Station X3T, 21/30
Station 9RF, 22/30
Station 700, 23/30
Station V2P, 24/30
Station 1EJ, 25/30
Station 8X9, 26/30
Station VML, 27/30
Station 4AH, 28/30
Station VV4, 29/30
Station KN2, 30/30


[I 2026-01-13 17:42:32,284] Trial 1 finished with value: 0.029789295218264063 and parameters: {'units': 128, 'learning_rate': 0.0005950773136670117, 'batch_size': 16, 'seq_len': 80}. Best is trial 1 with value: 0.029789295218264063.


...

[I 2026-01-13 18:29:03,280] Trial 17 finished with value: 0.024439921835437417 and parameters: {'units': 48, 'learning_rate': 0.004443272361307879, 'batch_size': 16, 'seq_len': 100}. Best is trial 11 with value: 0.02340293808374554.


Station CUF, 1/30
Station KQC, 2/30
Station J52, 3/30
Station XTG, 4/30
Station EOE, 5/30
Station O2O, 6/30
Station Y5A, 7/30
Station NZV, 8/30
Station 4RW, 9/30
Station 488, 10/30
Station WDP, 11/30
Station ESA, 12/30
Station GUQ, 13/30
Station 0Y9, 14/30
Station 0WC, 15/30
Station KHT, 16/30
Station FBH, 17/30
Station I9U, 18/30
Station 6WR, 19/30
Station 94K, 20/30
Station X3T, 21/30
Station 9RF, 22/30
Station 700, 23/30
Station V2P, 24/30
Station 1EJ, 25/30
Station 8X9, 26/30
Station VML, 27/30
Station 4AH, 28/30
Station VV4, 29/30
Station KN2, 30/30


[I 2026-01-13 18:33:31,483] Trial 18 finished with value: 0.04242063779383898 and parameters: {'units': 48, 'learning_rate': 0.00010416638560757307, 'batch_size': 16, 'seq_len': 120}. Best is trial 11 with value: 0.02340293808374554.


Station CUF, 1/30
Station KQC, 2/30
Station J52, 3/30
Station XTG, 4/30
Station EOE, 5/30
Station O2O, 6/30
Station Y5A, 7/30
Station NZV, 8/30
Station 4RW, 9/30
Station 488, 10/30
Station WDP, 11/30
Station ESA, 12/30
Station GUQ, 13/30
Station 0Y9, 14/30
Station 0WC, 15/30
Station KHT, 16/30
Station FBH, 17/30
Station I9U, 18/30
Station 6WR, 19/30
Station 94K, 20/30
Station X3T, 21/30
Station 9RF, 22/30
Station 700, 23/30
Station V2P, 24/30
Station 1EJ, 25/30
Station 8X9, 26/30
Station VML, 27/30
Station 4AH, 28/30
Station VV4, 29/30
Station KN2, 30/30


[I 2026-01-13 18:36:45,103] Trial 19 finished with value: 0.027982922550290824 and parameters: {'units': 80, 'learning_rate': 0.003653597251627767, 'batch_size': 64, 'seq_len': 110}. Best is trial 11 with value: 0.02340293808374554.


In [17]:
print("Best params found:", study.best_params)

Best params found: {'units': 64, 'learning_rate': 0.009172885794840763, 'batch_size': 16, 'seq_len': 120}
